<a href="https://colab.research.google.com/github/asanis/testcode/blob/main/TOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Εισαγωγή βιβλιοθηκών
import pandas as pd
import ipaddress
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Καταστολή FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Συνάρτηση για μετατροπή διευθύνσεων IP σε αριθμούς
def ip_to_int(ip):
    try:
        return int(ipaddress.ip_address(ip))
    except ValueError:
        return None  # Επιστρέφει None αν η διεύθυνση IP είναι άκυρη

In [ ]:

#  Φόρτωση δεδομένων
df = pd.read_csv('https://raw.githubusercontent.com/kdemertzis/EKPA/refs/heads/main/Data/DarkNet.csv', low_memory=False)

In [ ]:
# Μετατροπή των IP σε αριθμούς (Src_IP & Dst_IP)
df['Src_IP'] = df['Src_IP'].apply(ip_to_int)
df['Dst_IP'] = df['Dst_IP'].apply(ip_to_int)

In [ ]:
# Διαχωρισμός χαρακτηριστικών (features) και στόχου (target)
X = df.drop(columns=['Label-1', 'Label-2'])
y = df['Label-1']

In [ ]:
# Μετατροπή όλων των μη αριθμητικών τιμών σε NaN
X = X.apply(pd.to_numeric, errors='coerce')

In [ ]:
# Αντικατάσταση άπειρων τιμών με NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# Αντικατάσταση NaN τιμών με τη μέση τιμή κάθε στήλης
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [ ]:
# Κανονικοποίηση δεδομένων
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Διαχωρισμός σε εκπαίδευση και τεστ
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [ ]:
# Εξισορρόπηση κλάσεων με SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
# Δημιουργία και εκπαίδευση του Random Forest μοντέλου
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [ ]:
# Πρόβλεψη στα δεδομένα τεστ
y_pred = model.predict(X_test)

In [ ]:
# Αξιολόγηση του μοντέλου
accuracy = accuracy_score(y_test, y_pred)
print(f'🔹 Ακρίβεια Μοντέλου: {accuracy}')
print('\n🔹 Αναλυτική Αξιολόγηση:')
print(classification_report(y_test, y_pred))

🔹 Ακρίβεια Μοντέλου: 0.9966462525517643

🔹 Αναλυτική Αξιολόγηση:
              precision    recall  f1-score   support

     Non-Tor       1.00      1.00      1.00      9266
      NonVPN       0.99      1.00      1.00      6806
         Tor       0.95      0.91      0.93       425
         VPN       1.00      1.00      1.00      4077

    accuracy                           1.00     20574
   macro avg       0.99      0.98      0.98     20574
weighted avg       1.00      1.00      1.00     20574



In [ ]:
# Εμφάνιση του confusion matrix
print('\n🔹 Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))


🔹 Confusion Matrix:
[[9260    3    0    3]
 [   2 6785   18    1]
 [   0   38  387    0]
 [   0    1    3 4073]]
